In [86]:
import pandas as pd
from tqdm.notebook import tqdm

import gurobipy as gp
from gurobipy import GRB

from kcmc_instance import KCMC_Instance

K=4
M=3

df_instances = pd.read_parquet('/data/sample_K5M3_instances.parquet')
instance = df_instances.sample(1).iloc[0]['instance']
# print(str(instance))
instance = KCMC_Instance(instance, False, True, True)
print(instance)
instance = instance.to_single_sink(MAX_M=M)
print(instance)

L = [str(m) for m in range(M)]
A = instance.edges
P = instance.pois
I = instance.sensors

<KCMC:10:25:2:313:100:150 867868533 [0]>
<KCMC:10:25:1:313:100:150 867868533 [6]>


In [112]:
model = gp.Model('KCMC')



x_var = model.addVars(I, L, vtype=GRB.BINARY, name="x")

y_var = model.addVars(A, P, L, vtype=GRB.BINARY, name='y')


model.setObjective(x_var.sum('*', '*'), GRB.MINIMIZE)



# ilp_multi_disjunction = model.addConstrs(
#     (x_var.sum(i, '*') <= 1
#      for i in I),
#     name="ilp_multi_disjunction"
# )

# ilp_multi_flow = model.addConstrs(
#     ((y_var.sum(i, '*', p, l) - y_var.sum('*', i, p, l)) == 1
#      for i in set([comp for par in instance.edges for comp in par])
#      for p in P
#      for l in L
#      if i == f'p{p}'),
#     name="ilp_multi_flow_p"
# )

# ilp_multi_flow = model.addConstrs(
#     ((y_var.sum('k0', '*', p, l) - y_var.sum('*', 'k0', p, l)) == -1
#      for p in P
#      for l in L),
#     name="ilp_multi_flow_k"
# )

ilp_multi_flow = model.addConstrs(
    ((y_var.sum(i, '*', '*', '*') == y_var.sum('*', i, '*', '*'))
     for i in set([comp for par in instance.edges for comp in par])
     if i.startswith('s')),
    name="ilp_multi_flow_i"
)

# ilp_multi_projection = model.addConstrs(
#     (y_var.sum(f's{i}', '*', '*', l) <= x_var[i, l]
#      for i in I
#      for l in L),
#     name="ilp_multi_projection"
# )

# ilp_multi_k = model.addConstrs(
#     (x_var.sum(str(i), '*') >= 1
#      for p in P         
#      for i in instance.poi_sensor[int(p)]),
#     name="ilp_multi_k"
# )



model.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
[client] info  : Validate web license token 5b69dd03-2396-4717-873b-d22f1d57b301 for container cc597115d57941ccf0e9117f1309800d9e5ef2be4695a9d032c7bec1f87fb6dc
Academic license - for non-commercial use only - registered to josefrf@ufmg.br
[client] info  : Web license token valid
Optimize a model with 58 rows, 2673 columns and 2820 nonzeros
Model fingerprint: 0x63034611
Variable types: 0 continuous, 2673 integer (2673 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 6 rows and 2673 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds
Thread count was 1 (of 8 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


In [108]:
# x_var

y_var.select('*', 'k0', '*', '*')

[<gurobi.Var y[s25,k0,0,0] (value -0.0)>,
 <gurobi.Var y[s25,k0,0,1] (value -0.0)>,
 <gurobi.Var y[s25,k0,0,2] (value -0.0)>,
 <gurobi.Var y[s25,k0,1,0] (value -0.0)>,
 <gurobi.Var y[s25,k0,1,1] (value -0.0)>,
 <gurobi.Var y[s25,k0,1,2] (value -0.0)>,
 <gurobi.Var y[s25,k0,2,0] (value -0.0)>,
 <gurobi.Var y[s25,k0,2,1] (value -0.0)>,
 <gurobi.Var y[s25,k0,2,2] (value -0.0)>,
 <gurobi.Var y[s25,k0,3,0] (value -0.0)>,
 <gurobi.Var y[s25,k0,3,1] (value -0.0)>,
 <gurobi.Var y[s25,k0,3,2] (value -0.0)>,
 <gurobi.Var y[s25,k0,4,0] (value -0.0)>,
 <gurobi.Var y[s25,k0,4,1] (value -0.0)>,
 <gurobi.Var y[s25,k0,4,2] (value -0.0)>,
 <gurobi.Var y[s25,k0,5,0] (value -0.0)>,
 <gurobi.Var y[s25,k0,5,1] (value -0.0)>,
 <gurobi.Var y[s25,k0,5,2] (value -0.0)>,
 <gurobi.Var y[s25,k0,6,0] (value -0.0)>,
 <gurobi.Var y[s25,k0,6,1] (value -0.0)>,
 <gurobi.Var y[s25,k0,6,2] (value -0.0)>,
 <gurobi.Var y[s25,k0,7,0] (value -0.0)>,
 <gurobi.Var y[s25,k0,7,1] (value -0.0)>,
 <gurobi.Var y[s25,k0,7,2] (value 

In [110]:
y_var

{('p0', 's0', '0', '0'): <gurobi.Var y[p0,s0,0,0] (value -0.0)>,
 ('p0', 's0', '0', '1'): <gurobi.Var y[p0,s0,0,1] (value -0.0)>,
 ('p0', 's0', '0', '2'): <gurobi.Var y[p0,s0,0,2] (value -0.0)>,
 ('p0', 's0', '1', '0'): <gurobi.Var y[p0,s0,1,0] (value -0.0)>,
 ('p0', 's0', '1', '1'): <gurobi.Var y[p0,s0,1,1] (value -0.0)>,
 ('p0', 's0', '1', '2'): <gurobi.Var y[p0,s0,1,2] (value -0.0)>,
 ('p0', 's0', '2', '0'): <gurobi.Var y[p0,s0,2,0] (value -0.0)>,
 ('p0', 's0', '2', '1'): <gurobi.Var y[p0,s0,2,1] (value -0.0)>,
 ('p0', 's0', '2', '2'): <gurobi.Var y[p0,s0,2,2] (value -0.0)>,
 ('p0', 's0', '3', '0'): <gurobi.Var y[p0,s0,3,0] (value -0.0)>,
 ('p0', 's0', '3', '1'): <gurobi.Var y[p0,s0,3,1] (value -0.0)>,
 ('p0', 's0', '3', '2'): <gurobi.Var y[p0,s0,3,2] (value -0.0)>,
 ('p0', 's0', '4', '0'): <gurobi.Var y[p0,s0,4,0] (value -0.0)>,
 ('p0', 's0', '4', '1'): <gurobi.Var y[p0,s0,4,1] (value -0.0)>,
 ('p0', 's0', '4', '2'): <gurobi.Var y[p0,s0,4,2] (value -0.0)>,
 ('p0', 's0', '5', '0'): 

In [76]:
ilp_multi_flow

{('p5', '0', '0'): <gurobi.Constr ilp_multi_flow[p5,0,0]>,
 ('p5', '0', '1'): <gurobi.Constr ilp_multi_flow[p5,0,1]>,
 ('p5', '0', '2'): <gurobi.Constr ilp_multi_flow[p5,0,2]>,
 ('p5', '1', '0'): <gurobi.Constr ilp_multi_flow[p5,1,0]>,
 ('p5', '1', '1'): <gurobi.Constr ilp_multi_flow[p5,1,1]>,
 ('p5', '1', '2'): <gurobi.Constr ilp_multi_flow[p5,1,2]>,
 ('p5', '2', '0'): <gurobi.Constr ilp_multi_flow[p5,2,0]>,
 ('p5', '2', '1'): <gurobi.Constr ilp_multi_flow[p5,2,1]>,
 ('p5', '2', '2'): <gurobi.Constr ilp_multi_flow[p5,2,2]>,
 ('p5', '3', '0'): <gurobi.Constr ilp_multi_flow[p5,3,0]>,
 ('p5', '3', '1'): <gurobi.Constr ilp_multi_flow[p5,3,1]>,
 ('p5', '3', '2'): <gurobi.Constr ilp_multi_flow[p5,3,2]>,
 ('p5', '4', '0'): <gurobi.Constr ilp_multi_flow[p5,4,0]>,
 ('p5', '4', '1'): <gurobi.Constr ilp_multi_flow[p5,4,1]>,
 ('p5', '4', '2'): <gurobi.Constr ilp_multi_flow[p5,4,2]>,
 ('p5', '5', '0'): <gurobi.Constr ilp_multi_flow[p5,5,0]>,
 ('p5', '5', '1'): <gurobi.Constr ilp_multi_flow[p5,5,1]